# Démonstration : Protocoles Challenge-Response

Ce notebook implémente des protocoles d'authentification Challenge-Response pour éviter la transmission de mots de passe en clair sur le réseau.

## 📚 Concepts couverts

1. **Challenge-Response basique avec HMAC**
2. **Protocole SCRAM (Salted Challenge Response Authentication Mechanism)**
3. **Attaques et défenses**
4. **Comparaison avec authentification par mot de passe simple**

---

In [ ]:
import hmac
import hashlib
import os
import time
import base64
from typing import Tuple, Dict

## 1. Challenge-Response avec HMAC

### Principe

Le serveur envoie un **challenge** (nombre aléatoire) au client. Le client calcule une **réponse** en utilisant HMAC avec sa clé secrète (dérivée du mot de passe) et le challenge.

**Avantages** :
- Le mot de passe n'est jamais transmis sur le réseau
- Protection contre les attaques par rejeu (le challenge change à chaque authentification)

**Protocole** :
```
Client → Serveur : username
Serveur → Client : challenge (random)
Client → Serveur : HMAC(password, challenge)
Serveur : Vérifie HMAC avec le password stocké
```

In [ ]:
class SimpleAuthServer:
    """Serveur d'authentification Challenge-Response avec HMAC"""
    
    def __init__(self):
        # Base de données simulée : {username: password_key}
        self.users_db = {}
        # Challenges actifs : {username: (challenge, timestamp)}
        self.active_challenges = {}
        self.challenge_timeout = 60  # 60 secondes
    
    def register_user(self, username: str, password: str):
        """Enregistre un utilisateur avec son mot de passe"""
        # En pratique, on stockerait un hash du password, pas le password lui-même
        # Ici on simplifie pour la démonstration
        password_key = hashlib.sha256(password.encode()).digest()
        self.users_db[username] = password_key
        print(f"✅ Utilisateur '{username}' enregistré")
    
    def generate_challenge(self, username: str) -> bytes:
        """Génère un challenge pour un utilisateur"""
        if username not in self.users_db:
            raise ValueError(f"Utilisateur '{username}' inconnu")
        
        # Génère un challenge aléatoire de 32 bytes
        challenge = os.urandom(32)
        self.active_challenges[username] = (challenge, time.time())
        
        print(f"🔐 Challenge généré pour '{username}': {challenge.hex()[:32]}...")
        return challenge
    
    def verify_response(self, username: str, response: bytes) -> bool:
        """Vérifie la réponse du client"""
        if username not in self.active_challenges:
            print("❌ Aucun challenge actif pour cet utilisateur")
            return False
        
        challenge, timestamp = self.active_challenges[username]
        
        # Vérifie le timeout
        if time.time() - timestamp > self.challenge_timeout:
            print("❌ Challenge expiré")
            del self.active_challenges[username]
            return False
        
        # Calcule la réponse attendue
        password_key = self.users_db[username]
        expected_response = hmac.new(password_key, challenge, hashlib.sha256).digest()
        
        # Compare de manière sécurisée (timing-safe)
        valid = hmac.compare_digest(response, expected_response)
        
        # Supprime le challenge utilisé
        del self.active_challenges[username]
        
        if valid:
            print(f"✅ Authentification réussie pour '{username}'")
        else:
            print(f"❌ Authentification échouée pour '{username}'")
        
        return valid

In [ ]:
class SimpleAuthClient:
    """Client d'authentification Challenge-Response"""
    
    def __init__(self, username: str, password: str):
        self.username = username
        self.password_key = hashlib.sha256(password.encode()).digest()
    
    def compute_response(self, challenge: bytes) -> bytes:
        """Calcule la réponse au challenge"""
        response = hmac.new(self.password_key, challenge, hashlib.sha256).digest()
        print(f"🔑 Réponse calculée: {response.hex()[:32]}...")
        return response

In [ ]:
# Démonstration du protocole Challenge-Response

print("=" * 60)
print("DÉMONSTRATION: Challenge-Response avec HMAC")
print("=" * 60)

# Initialisation
server = SimpleAuthServer()
server.register_user("alice", "SecurePassword123!")

print("\n--- Authentification réussie ---")
# Simulation d'une authentification réussie
client = SimpleAuthClient("alice", "SecurePassword123!")
challenge = server.generate_challenge("alice")
response = client.compute_response(challenge)
server.verify_response("alice", response)

print("\n--- Authentification échouée (mauvais mot de passe) ---")
# Simulation d'une authentification échouée
attacker = SimpleAuthClient("alice", "WrongPassword")
challenge = server.generate_challenge("alice")
response = attacker.compute_response(challenge)
server.verify_response("alice", response)

print("\n--- Attaque par rejeu (replay attack) ---")
# Simulation d'une attaque par rejeu
client = SimpleAuthClient("alice", "SecurePassword123!")
challenge = server.generate_challenge("alice")
response = client.compute_response(challenge)
server.verify_response("alice", response)  # Première tentative réussie

print("\nTentative de rejeu de la même réponse...")
# L'attaquant essaie de rejouer la même réponse
result = server.verify_response("alice", response)  # Échoue car challenge déjà utilisé

## 2. Protocole SCRAM (Salted Challenge Response Authentication Mechanism)

SCRAM est un protocole d'authentification standardisé (RFC 5802) qui améliore le Challenge-Response en ajoutant :

1. **Salted Password Storage** : Le serveur ne stocke pas le mot de passe, mais `SaltedPassword = PBKDF2(password, salt, iterations)`
2. **Mutual Authentication** : Le client vérifie aussi l'identité du serveur
3. **Protection contre les attaques** : Utilise des nonces (client et serveur)

**Protocole simplifié** :
```
1. Client → Serveur : username, client_nonce
2. Serveur → Client : server_nonce, salt, iterations
3. Client calcule SaltedPassword = PBKDF2(password, salt, iterations)
4. Client → Serveur : ClientProof (HMAC avec SaltedPassword)
5. Serveur vérifie ClientProof
6. Serveur → Client : ServerSignature (preuve que le serveur connaît le secret)
7. Client vérifie ServerSignature
```

In [ ]:
class SCRAMServer:
    """Serveur d'authentification SCRAM simplifié"""
    
    def __init__(self, iterations=4096):
        # Base de données : {username: (salt, stored_key, server_key)}
        self.users_db = {}
        self.iterations = iterations
        self.active_sessions = {}
    
    def register_user(self, username: str, password: str):
        """Enregistre un utilisateur avec PBKDF2"""
        salt = os.urandom(16)
        
        # Dérive la clé avec PBKDF2
        salted_password = hashlib.pbkdf2_hmac(
            'sha256', password.encode(), salt, self.iterations
        )
        
        # Calcule les clés stockées
        client_key = hmac.new(salted_password, b"Client Key", hashlib.sha256).digest()
        stored_key = hashlib.sha256(client_key).digest()
        server_key = hmac.new(salted_password, b"Server Key", hashlib.sha256).digest()
        
        self.users_db[username] = (salt, stored_key, server_key)
        print(f"✅ Utilisateur '{username}' enregistré (SCRAM)")
        print(f"   Salt: {salt.hex()[:32]}...")
        print(f"   Iterations: {self.iterations}")
    
    def start_auth(self, username: str, client_nonce: str) -> Tuple[str, bytes, int]:
        """Démarre l'authentification et retourne les paramètres"""
        if username not in self.users_db:
            raise ValueError(f"Utilisateur '{username}' inconnu")
        
        server_nonce = base64.b64encode(os.urandom(16)).decode('utf-8')
        nonce = client_nonce + server_nonce
        
        salt, stored_key, server_key = self.users_db[username]
        
        # Stocke la session
        self.active_sessions[username] = {
            'nonce': nonce,
            'stored_key': stored_key,
            'server_key': server_key,
            'timestamp': time.time()
        }
        
        print(f"🔐 Session SCRAM démarrée pour '{username}'")
        print(f"   Server nonce: {server_nonce[:16]}...")
        
        return server_nonce, salt, self.iterations
    
    def verify_proof(self, username: str, client_proof: bytes) -> Tuple[bool, bytes]:
        """Vérifie la preuve du client et génère la signature du serveur"""
        if username not in self.active_sessions:
            return False, b""
        
        session = self.active_sessions[username]
        
        # Calcule AuthMessage (simplifié)
        auth_message = f"{username},{session['nonce']}".encode()
        
        # Calcule ClientSignature
        client_signature = hmac.new(
            session['stored_key'], auth_message, hashlib.sha256
        ).digest()
        
        # Récupère ClientKey depuis ClientProof
        client_key = bytes(a ^ b for a, b in zip(client_proof, client_signature))
        
        # Vérifie que hash(ClientKey) == StoredKey
        computed_stored_key = hashlib.sha256(client_key).digest()
        valid = hmac.compare_digest(computed_stored_key, session['stored_key'])
        
        if valid:
            # Génère ServerSignature pour mutual authentication
            server_signature = hmac.new(
                session['server_key'], auth_message, hashlib.sha256
            ).digest()
            print(f"✅ SCRAM: Client authentifié pour '{username}'")
            del self.active_sessions[username]
            return True, server_signature
        else:
            print(f"❌ SCRAM: Échec d'authentification pour '{username}'")
            return False, b""

In [ ]:
class SCRAMClient:
    """Client d'authentification SCRAM"""
    
    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password
        self.client_nonce = base64.b64encode(os.urandom(16)).decode('utf-8')
    
    def compute_proof(self, server_nonce: str, salt: bytes, iterations: int) -> Tuple[bytes, str]:
        """Calcule la preuve client et stocke les infos pour vérification serveur"""
        nonce = self.client_nonce + server_nonce
        
        # Calcule SaltedPassword
        salted_password = hashlib.pbkdf2_hmac(
            'sha256', self.password.encode(), salt, iterations
        )
        
        # Calcule ClientKey et StoredKey
        client_key = hmac.new(salted_password, b"Client Key", hashlib.sha256).digest()
        stored_key = hashlib.sha256(client_key).digest()
        
        # Calcule AuthMessage (simplifié)
        auth_message = f"{self.username},{nonce}".encode()
        
        # Calcule ClientSignature
        client_signature = hmac.new(stored_key, auth_message, hashlib.sha256).digest()
        
        # ClientProof = ClientKey XOR ClientSignature
        client_proof = bytes(a ^ b for a, b in zip(client_key, client_signature))
        
        # Stocke ServerKey pour vérification mutuelle
        server_key = hmac.new(salted_password, b"Server Key", hashlib.sha256).digest()
        self.expected_server_signature = hmac.new(
            server_key, auth_message, hashlib.sha256
        ).digest()
        
        print(f"🔑 Client proof calculée: {client_proof.hex()[:32]}...")
        
        return client_proof, nonce
    
    def verify_server(self, server_signature: bytes) -> bool:
        """Vérifie que le serveur connaît le secret (mutual authentication)"""
        valid = hmac.compare_digest(server_signature, self.expected_server_signature)
        
        if valid:
            print(f"✅ Serveur authentifié (mutual authentication)")
        else:
            print(f"❌ ATTENTION: Le serveur n'a pas pu être authentifié!")
        
        return valid

In [ ]:
# Démonstration du protocole SCRAM

print("=" * 60)
print("DÉMONSTRATION: SCRAM (Mutual Authentication)")
print("=" * 60)

# Initialisation
scram_server = SCRAMServer(iterations=4096)
scram_server.register_user("bob", "MySecretPassword456!")

print("\n--- Authentification mutuelle réussie ---")
# Phase 1: Client démarre l'authentification
scram_client = SCRAMClient("bob", "MySecretPassword456!")
print(f"👤 Client nonce: {scram_client.client_nonce[:16]}...")

# Phase 2: Serveur répond avec ses paramètres
server_nonce, salt, iterations = scram_server.start_auth("bob", scram_client.client_nonce)

# Phase 3: Client calcule et envoie sa preuve
client_proof, nonce = scram_client.compute_proof(server_nonce, salt, iterations)

# Phase 4 & 5: Serveur vérifie et retourne sa signature
auth_success, server_signature = scram_server.verify_proof("bob", client_proof)

# Phase 6: Client vérifie le serveur
if auth_success:
    scram_client.verify_server(server_signature)

print("\n--- Authentification échouée (mauvais mot de passe) ---")
# Tentative avec mauvais mot de passe
bad_client = SCRAMClient("bob", "WrongPassword")
server_nonce, salt, iterations = scram_server.start_auth("bob", bad_client.client_nonce)
client_proof, nonce = bad_client.compute_proof(server_nonce, salt, iterations)
auth_success, server_signature = scram_server.verify_proof("bob", client_proof)

## 3. Comparaison des approches

Comparons différentes méthodes d'authentification :

In [ ]:
import pandas as pd

comparison = {
    "Méthode": [
        "Plaintext Password",
        "Hashed Password (SHA-256)",
        "Challenge-Response (HMAC)",
        "SCRAM"
    ],
    "Mot de passe sur réseau": ["❌ Oui", "❌ Oui (hash)", "✅ Non", "✅ Non"],
    "Protection replay": ["❌ Non", "❌ Non", "✅ Oui", "✅ Oui"],
    "Mutual Auth": ["❌ Non", "❌ Non", "❌ Non", "✅ Oui"],
    "Stockage serveur": ["Plaintext", "Hash", "Hash/Key", "Salted Hash"],
    "Résistance rainbow table": ["❌ N/A", "❌ Faible", "✅ Bonne", "✅ Excellente"],
    "Complexité": ["Faible", "Faible", "Moyenne", "Élevée"]
}

df = pd.DataFrame(comparison)
print("\n" + "=" * 80)
print("COMPARAISON DES MÉTHODES D'AUTHENTIFICATION")
print("=" * 80)
print(df.to_string(index=False))

## 4. Simulation d'attaques

### 4.1 Attaque par rejeu (Replay Attack)

In [ ]:
print("=" * 60)
print("SIMULATION: Attaque par rejeu")
print("=" * 60)

# Sans protection (mot de passe en clair)
print("\n--- Sans protection (mot de passe transmis en clair) ---")
captured_password = "SecurePassword123!"
print(f"🕵️ Attaquant capture: {captured_password}")
print("❌ L'attaquant peut rejouer ce mot de passe indéfiniment!")

# Avec Challenge-Response
print("\n--- Avec Challenge-Response ---")
server = SimpleAuthServer()
server.register_user("victim", "SecurePassword123!")

# Authentification légitime
client = SimpleAuthClient("victim", "SecurePassword123!")
challenge1 = server.generate_challenge("victim")
response1 = client.compute_response(challenge1)
print(f"🕵️ Attaquant capture la réponse: {response1.hex()[:32]}...")

# L'authentification réussit
server.verify_response("victim", response1)

# L'attaquant essaie de rejouer la réponse capturée
print("\n🎭 L'attaquant tente de rejouer la réponse capturée...")
challenge2 = server.generate_challenge("victim")
print("❌ Mais le challenge a changé! La réponse capturée ne fonctionne plus:")
server.verify_response("victim", response1)  # Échoue

## 📊 Résumé des bonnes pratiques

### ✅ À FAIRE

1. **Utiliser Challenge-Response** pour éviter la transmission de mots de passe
2. **Implémenter SCRAM** pour l'authentification mutuelle (client ET serveur)
3. **Ajouter des timeouts** aux challenges (expiration après X secondes)
4. **Utiliser PBKDF2/bcrypt** pour le stockage côté serveur
5. **Valider avec `hmac.compare_digest()`** pour éviter les timing attacks
6. **Utiliser des nonces** (client et serveur) pour unicité

### ❌ À NE PAS FAIRE

1. ❌ Transmettre le mot de passe en clair sur le réseau
2. ❌ Réutiliser les challenges (vulnérable aux replay attacks)
3. ❌ Stocker les mots de passe en plaintext côté serveur
4. ❌ Utiliser des comparaisons simples (`==`) pour vérifier les MACs
5. ❌ Omettre l'authentification mutuelle dans les contextes sensibles

---

## 🔗 Références

- **RFC 5802** : Salted Challenge Response Authentication Mechanism (SCRAM)
- **RFC 2104** : HMAC: Keyed-Hashing for Message Authentication
- **OWASP** : Authentication Cheat Sheet
- **Challenge-Response Authentication** : https://en.wikipedia.org/wiki/Challenge–response_authentication